# Text Classification with TensorFlow, Keras, and Cleanlab


In this quick-start tutorial, we use `cleanlab` to find potential label errors in the IMDb movie review text classification dataset. This dataset contains 50,000 text reviews, each labeled with a binary sentiment polarity label indicating whether the review is positive (1) or negative (0). `cleanlab` will shortlist _hundreds_ of examples that confuse our ML model the most; many of which are potential label errors, edge cases, or otherwise ambiguous examples.

**Overview of what we'll do in this tutorial:**

- Build a simple TensorFlow & Keras neural net and wrap it with SciKeras to make it scikit-learn compatible.

- Use this classifier to compute out-of-sample predicted probabilities, `pred_probs`, via cross validation.

- Identify potential label errors in the data with `cleanlab`'s `find_label_issues` method.

- Train a more robust version of the same neural net via `cleanlab`'s `LearningWithNoisyLabels` wrapper.

**Data:** https://ai.stanford.edu/~amaas/data/sentiment/


## **1. Install required dependencies**


Install the following dependencies with `pip install`:

1. cleanlab
2. sklearn
3. pandas
4. tensorflow
5. tensorflow-datasets
6. scikeras


In [1]:
dependencies = ["cleanlab", "sklearn", "pandas", "tensorflow", "tensorflow_datasets", "scikeras"]

if "google.colab" in str(get_ipython()):  # Check if it's running in Google Colab
    %pip install git+https://github.com/weijinglok/cleanlab.git@ca8368c4c0088c755c5930cfc7c3b9e52a9c7945
    cmd = ' '.join([dep for dep in dependencies if dep != "cleanlab"])
    %pip install $cmd
else:
    missing_dependencies = []
    for dependency in dependencies:
        try:
            __import__(dependency)
        except ImportError:
            missing_dependencies.append(dependency)

    if len(missing_dependencies) > 0:
        print("Missing required dependencies:")
        print(*missing_dependencies, sep=", ")
        print("\nPlease install them before running the rest of this notebook.")

2022-04-05 10:27:36.759305: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hostedtoolcache/Python/3.8.12/x64/lib
2022-04-05 10:27:36.759341: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import os
import random
import numpy as np

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Controls amount of tensorflow output

SEED = 123456  # Just for reproducibility

np.random.seed(SEED)
random.seed(SEED)


## **2. Load and preprocess the IMDb text dataset**


This dataset is provided in TensorFlow's Datasets.


In [3]:
%%capture

import tensorflow_datasets as tfds

raw_full_ds = tfds.load(
    name="imdb_reviews", split=("train+test"), batch_size=-1, as_supervised=True
)
raw_full_texts, full_labels = tfds.as_numpy(raw_full_ds)


2022-04-05 10:27:40.102458: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


2022-04-05 10:28:33.538475: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hostedtoolcache/Python/3.8.12/x64/lib
2022-04-05 10:28:33.538525: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-05 10:28:33.538549: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fv-az198-711): /proc/driver/nvidia/version does not exist
2022-04-05 10:28:33.540080: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Let's print the first example.

In [4]:
i = 0
print(f"Example Label: {full_labels[i]}")
print(f"Example Text: {raw_full_texts[i]}")


Example Label: 0
Example Text: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."


The data are stored as two numpy arrays:

1. `raw_full_texts` for the movie reviews in text format,
2. `full_labels` for the labels.


<div class="alert alert-info">
Bringing Your Own Data (BYOD)?

You can easily replace the above with your own tabular dataset, and continue with the rest of the tutorial.

</div>


Define a function to preprocess the text data by:

1. Converting it to lower case
2. Removing the HTML break tags: `<br />`
3. Removing any punctuation marks


In [5]:
import tensorflow as tf
import re
import string


def preprocess_text(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(stripped_html, f"[{re.escape(string.punctuation)}]", "")


We use a `TextVectorization` layer to preprocess, tokenize, and vectorize our text data, thus making it suitable as input for a neural network.


In [6]:
from tensorflow.keras import layers

tf.keras.utils.set_random_seed(SEED)

max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=preprocess_text,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)


Adapting `vectorize_layer` to the text data creates a mapping of each token (i.e. word) to an integer index. Subsequently, we can vectorize our text data by using this mapping. Finally, we'll also convert our text data into a numpy array as required by `cleanlab`.


In [7]:
%%capture

vectorize_layer.adapt(raw_full_texts)
full_texts = vectorize_layer(raw_full_texts)
full_texts = full_texts.numpy()


## **3. Define a classification model and compute out-of-sample predicted probabilities**


Here, we build a simple neural network for classification with TensorFlow and Keras.


In [8]:
from tensorflow.keras import losses, metrics


def get_net():
    net = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(None,), dtype="int64"),
            layers.Embedding(max_features + 1, 16),
            layers.Dropout(0.2),
            layers.GlobalAveragePooling1D(),
            layers.Dropout(0.2),
            layers.Dense(1),
        ]
    )  # outputs probability that text belongs to class 1

    net.compile(
        optimizer="adam",
        loss=losses.BinaryCrossentropy(from_logits=True),
        metrics=metrics.BinaryAccuracy(),
    )
    return net


As some of `cleanlab`'s feature requires scikit-learn compatibility, we will need to adapt the above TensorFlow & Keras neural net accordingly. [SciKeras](https://www.adriangb.com/scikeras/stable/) is a convenient package that makes this really easy.


In [9]:
from scikeras.wrappers import KerasClassifier

model = KerasClassifier(get_net(), epochs=10)


To identify label issues, cleanlab requires a probabilistic prediction from your model for every datapoint that should be considered. However these predictions will be _overfit_ (and thus unreliable) for datapoints the model was previously trained on. `cleanlab` is intended to only be used with **out-of-sample** predicted probabilities, i.e. on datapoints held-out from the model during the training.

K-fold cross-validation is a straightforward way to produce out-of-sample predicted probabilites for every datapoint in the dataset, by training K copies of our model on different data subsets and using each copy to predict on the subset of data it did not see during training. We can obtain cross-validated out-of-sample predicted probabilities from any classifier via a scikit-learn simple wrapper:


In [10]:
from sklearn.model_selection import cross_val_predict

num_crossval_folds = 3  # chosen for efficiency here, values like 5 or 10 will generally work better
pred_probs = cross_val_predict(
    model, full_texts, full_labels, cv=num_crossval_folds, method="predict_proba"
)


2022-04-05 10:28:44.436275: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram:///tmp/tmpzk6ay0_z/assets


Epoch 1/10


   1/1042 [..............................] - ETA: 12:41 - loss: 0.6933 - binary_accuracy: 0.4688

  11/1042 [..............................] - ETA: 5s - loss: 0.6935 - binary_accuracy: 0.4972   

...

372/521 [====================>.........] - ETA: 0s

408/521 [======================>.......] - ETA: 0s

445/521 [========================>.....] - ETA: 0s

483/521 [==========================>...] - ETA: 0s

521/521 [==============================] - 1s 1ms/step


An additional benefit of cross-validation is that it facilitates more reliable evaluation of our model than a single training/validation split.


In [11]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(full_labels, pred_probs[:, 1])
print(f"Cross-validated estimate of held-out AUC score: {auc}")


Cross-validated estimate of held-out AUC score: 0.9522547736


## **4. Use cleanlab to find potential label errors**


Based on the given labels and out-of-sample predicted probabilities, `cleanlab` can quickly help us identify label issues. Here we request that the indices of the identified label issues should be sorted by `cleanlab`'s self-confidence score, which measures the quality of each given label via the probability assigned it in our model's prediction.


In [12]:
from cleanlab.filter import find_label_issues

ranked_label_issues = find_label_issues(
    labels=full_labels, pred_probs=pred_probs, return_indices_ranked_by="self_confidence"
)


Let's review some of the most likely label errors:


In [13]:
print(
    f"cleanlab found {len(ranked_label_issues)} potential label errors. Here are indices of the top 10 most likely errors: \n {ranked_label_issues[:10]}"
)


cleanlab found 1034 potential label errors. Here are indices of the top 10 most likely errors: 
 [10404 44582 43777 30151 16633 21348 17701   434 13912 30342]


To help us inspect these datapoints, we define a method to print any example from the dataset. We then display some of the top-ranked label issues identified by `cleanlab`:


In [14]:
import pandas as pd

pd.set_option("display.max_colwidth", None)


def print_as_df(index):
    return pd.DataFrame({"texts": raw_full_texts[index], "labels": full_labels[index]}, [index])


Here's a review labeled as positive (1), but it should be negative (0).
Some noteworthy snippets extracted from the review text:

> - "...incredibly **awful** score..."
>
> - "...**worst** Foley work ever done."
>
> - "...script is **incomprehensible**..."
>
> - "...editing is just **bizarre**."
>
> - "...**atrocious** pan and scan..."
>
> - "...**incoherent mess**..."
>
> - "...**amateur** directing there."


In [15]:
print_as_df(44582)


,texts,labels
44582,"b'This movie is stuffed full of stock Horror movie goodies: chained lunatics, pre-meditated murder, a mad (vaguely lesbian) female scientist with an even madder father who wears a mask because of his horrible disfigurement, poisoning, spooky castles, werewolves (male and female), adultery, slain lovers, Tibetan mystics, the half-man/half-plant victim of some unnamed experiment, grave robbing, mind control, walled up bodies, a car crash on a lonely road, electrocution, knights in armour - the lot, all topped off with an incredibly awful score and some of the worst Foley work ever done.<br /><br />The script is incomprehensible (even by badly dubbed Spanish Horror movie standards) and some of the editing is just bizarre. In one scene where the lead female evil scientist goes to visit our heroine in her bedroom for one of the badly dubbed: ""That is fantastical. I do not understand. Explain to me again how this is..."" exposition scenes that litter this movie, there is a sudden hand held cutaway of the girl\'s thighs as she gets out of bed for no apparent reason at all other than to cover a cut in the bad scientist\'s ""Mwahaha! All your werewolfs belong mine!"" speech. Though why they went to the bother I don\'t know because there are plenty of other jarring jump cuts all over the place - even allowing for the atrocious pan and scan of the print I saw.<br /><br />The Director was, according to one interview with the star, drunk for most of the shoot and the film looks like it. It is an incoherent mess. It\'s made even more incoherent by the inclusion of werewolf rampage footage from a different film The Mark of the Wolf Man (made 4 years earlier, featuring the same actor but playing the part with more aggression and with a different shirt and make up - IS there a word in Spanish for ""Continuity""?) and more padding of another actor in the wolfman get-up ambling about in long shot.<br /><br />The music is incredibly bad varying almost at random from full orchestral creepy house music, to bosannova, to the longest piano and gong duet ever recorded. (Thinking about it, it might not have been a duet. It might have been a solo. The piano part was so simple it could have been picked out with one hand while the player whacked away at the gong with the other.) <br /><br />This is one of the most bewilderedly trance-state inducing bad movies of the year so far for me. Enjoy.<br /><br />Favourite line: ""Ilona! This madness and perversity will turn against you!"" How true.<br /><br />Favourite shot: The lover, discovering his girlfriend slain, dropping the candle in a cartoon-like demonstration of surprise. Rank amateur directing there.'",1


Here's a review labeled as positive (1), but it should be negative (0).
Some noteworthy snippets extracted from the review text:

> - "...film seems **cheap**."
>
> - "...unbelievably **bad**..."
>
> - "...cinematography is **badly** lit..."
>
> - "...everything looking **grainy** and **ugly**."
>
> - "...sound is so **terrible**..."


In [16]:
print_as_df(10404)


,texts,labels
10404,"b'This low-budget erotic thriller that has some good points, but a lot more bad one. The plot revolves around a female lawyer trying to clear her lover who is accused of murdering his wife. Being a soft-core film, that entails her going undercover at a strip club and having sex with possible suspects. As plots go for this type of genre, not to bad. The script is okay, and the story makes enough sense for someone up at 2 AM watching this not to notice too many plot holes. But everything else in the film seems cheap. The lead actors aren\'t that bad, but pretty much all the supporting ones are unbelievably bad (one girl seems like she is drunk and/or high). The cinematography is badly lit, with everything looking grainy and ugly. The sound is so terrible that you can barely hear what people are saying. The worst thing in this movie is the reason you\'re watching it-the sex. The reason people watch these things is for hot sex scenes featuring really hot girls in Red Shoe Diary situations. The sex scenes aren\'t hot they\'re sleazy, shot in that porno style where everything is just a master shot of two people going at it. The woman also look like they are refuges from a porn shoot. I\'m not trying to be rude or mean here, but they all have that breast implants and a burned out/weathered look. Even the title, ""Deviant Obsession"", sounds like a Hardcore flick. Not that I don\'t have anything against porn - in fact I love it. But I want my soft-core and my hard-core separate. What ever happened to actresses like Shannon Tweed, Jacqueline Lovell, Shannon Whirry and Kim Dawson? Women that could act and who would totally arouse you? And what happened to B erotic thrillers like Body Chemistry, Nighteyes and even Stripped to Kill. Sure, none of these where masterpieces, but at least they felt like movies. Plus, they were pushing the envelope, going beyond Hollywood\'s relatively prude stance on sex, sexual obsessions and perversions. Now they just make hard-core films without the hard-core sex.'",1


Here's a review labeled as positive (1), but it should be negative (0).
Some noteworthy snippets extracted from the review text:

> - "...hard to imagine a **boring** shark movie..."
>
> - "**Poor focus** in some scenes made the production seems **amateurish**."
>
> - "...**do nothing** to take advantage of..."
>
> - "...**far too few** scenes of any depth or variety."
>
> - "...just **look flat**...no contrast of depth..."
>
> - "...**introspective** and **dull**...constant **disappointment**."


In [17]:
print_as_df(30151)


,texts,labels
30151,"b'Like the gentle giants that make up the latter half of this film\'s title, Michael Oblowitz\'s latest production has grace, but it\'s also slow and ponderous. The producer\'s last outing, ""Mosquitoman-3D"" had the same problem. It\'s hard to imagine a boring shark movie, but they somehow managed it. The only draw for Hammerhead: Shark Frenzy was it\'s passable animatronix, which is always fun when dealing with wondrous worlds beneath the ocean\'s surface. But even that was only passable. Poor focus in some scenes made the production seems amateurish. With Dolphins and Whales, the technology is all but wasted. Cloudy scenes and too many close-ups of the film\'s giant subjects do nothing to take advantage of IMAX\'s stunning 3D capabilities. There are far too few scenes of any depth or variety. Close-ups of these awesome creatures just look flat and there is often only one creature in the cameras field, so there is no contrast of depth. Michael Oblowitz is trying to follow in his father\'s footsteps, but when you\'ve got Shark-Week on cable, his introspective and dull treatment of his subjects is a constant disappointment.'",1


`cleanlab` has shortlisted the most likely label errors to speed up your data cleaning process. With this list, you can decide whether to fix these label issues or remove ambiguous examples from the dataset.


## **5. Train a more robust model from noisy labels**


Fixing the label issues manually may be time-consuming, but at least `cleanlab` can filter these noisy examples and train a model on the remaining clean data for you automatically.
To demonstrate this, we first reload the dataset, this time with separate train and test splits.


In [18]:
raw_train_ds = tfds.load(name="imdb_reviews", split="train", batch_size=-1, as_supervised=True)
raw_test_ds = tfds.load(name="imdb_reviews", split="test", batch_size=-1, as_supervised=True)

raw_train_texts, train_labels = tfds.as_numpy(raw_train_ds)
raw_test_texts, test_labels = tfds.as_numpy(raw_test_ds)


We featurize the raw text using the same `vectorize_layer` as before, but first, reset its state and adapt it only on the train set (as is proper ML practice). We finally convert the vectorized text data in the train/test sets into numpy arrays.


In [19]:
vectorize_layer.reset_state()
vectorize_layer.adapt(raw_train_texts)

train_texts = vectorize_layer(raw_train_texts)
test_texts = vectorize_layer(raw_test_texts)

train_texts = train_texts.numpy()
test_texts = test_texts.numpy()


Let's now train and evaluate our original neural network model.


In [20]:
from sklearn.metrics import accuracy_score

model = KerasClassifier(get_net(), epochs=10)
model.fit(train_texts, train_labels)

preds = model.predict(test_texts)
acc = accuracy_score(test_labels, preds)
print(f"\n Test acuracy of original neural net: {acc}")


Epoch 1/10


  1/782 [..............................] - ETA: 8:28 - loss: 0.6940 - binary_accuracy: 0.6250

 10/782 [..............................] - ETA: 4s - loss: 0.6925 - binary_accuracy: 0.4875  

 21/782 [..............................] - ETA: 3s - loss: 0.6923 - binary_accuracy: 0.4762

 33/782 [>.............................] - ETA: 3s - loss: 0.6921 - binary_accuracy: 0.4773

...

672/782 [========================>.....] - ETA: 0s

714/782 [==========================>...] - ETA: 0s

748/782 [===========================>..] - ETA: 0s

782/782 [==============================] - 1s 1ms/step



 Test acuracy of original neural net: 0.86384


`cleanlab` provides a wrapper class that can easily be applied to any scikit-learn compatible model. Once wrapped, the resulting model can still be used in the exact same manner, but it will now train more robustly if the data have noisy labels.


In [21]:
from cleanlab.classification import LearningWithNoisyLabels

model = KerasClassifier(get_net(), epochs=10)  # Note we first re-instantiate the model
lnl = LearningWithNoisyLabels(clf=model, seed=SEED)  # lnl has same methods/attributes as model


When we train the `cleanlab`-wrapped model, the following operations take place: The original model is trained in a cross-validated fashion to produce out-of-sample predicted probabilities. Then, these predicted probabilites are used to identify label issues, which are then removed from the dataset. Finally, the original model is trained once more on the remaining clean subset of the data.


In [22]:
_ = lnl.fit(train_texts, train_labels)


Computing out of sample predicted probabilites via 5-fold cross validation. May take a while ...


INFO:tensorflow:Assets written to: ram:///tmp/tmpo_c_sbcy/assets


Epoch 1/10


  1/625 [..............................] - ETA: 5:10 - loss: 0.6925 - binary_accuracy: 0.5312

 14/625 [..............................] - ETA: 2s - loss: 0.6934 - binary_accuracy: 0.5134  

...

724/762 [===========================>..] - ETA: 0s - loss: 0.1555 - binary_accuracy: 0.9375

736/762 [===========================>..] - ETA: 0s - loss: 0.1550 - binary_accuracy: 0.9378

749/762 [============================>.] - ETA: 0s - loss: 0.1548 - binary_accuracy: 0.9378

760/762 [============================>.] - ETA: 0s - loss: 0.1547 - binary_accuracy: 0.9380

762/762 [==============================] - 3s 4ms/step - loss: 0.1548 - binary_accuracy: 0.9379


We can get predictions from the resulting cleanlab model and evaluate them, just like we did for our original neural network.


In [23]:
pred_labels = lnl.predict(test_texts)
acc = accuracy_score(test_labels, pred_labels)
print(f"Test acuracy of cleanlab's neural net: {acc}")


  1/782 [..............................] - ETA: 37s

 39/782 [>.............................] - ETA: 0s 

 80/782 [==>...........................] - ETA: 0s

118/782 [===>..........................] - ETA: 0s

158/782 [=====>........................] - ETA: 0s

...

679/782 [=========================>....] - ETA: 0s

720/782 [==========================>...] - ETA: 0s

762/782 [============================>.] - ETA: 0s

782/782 [==============================] - 1s 1ms/step


Test acuracy of cleanlab's neural net: 0.8668


We can see that the test set accuracy slightly improved as a result of the data cleaning. Note that this will not always be the case, especially when we are evaluating on test data that are themselves noisy. The best practice is to run `cleanlab` to identify potential label issues and then manually review them, before blindly trusting any accuracy metrics. In particular, the most effort should be made to ensure high-quality test data, which is supposed to reflect the expected performance of our model during deployment.
